# ***Lecture Datasets***

In [ ]:
import math
import time
def calculate_distance(coord1, coord2):
    return math.sqrt((coord1[0] - coord2[0])**2 + (coord1[1] - coord2[1])**2)

def read_tsp_file(file_path):
    coordinates = {}
    with open(file_path, 'r') as file:
        lines = file.readlines()
        node_coord_section_index = lines.index('NODE_COORD_SECTION\n')
        for line in lines[node_coord_section_index + 1:]:
            if line.strip() == 'EOF':
                break
            parts = line.strip().split()
            node_id = int(parts[0])
            x, y = map(float, parts[1:])
            coordinates[node_id] = (x, y)
    return coordinates


def create_distance_matrix(coordinates):
    num_cities = len(coordinates)
    distance_matrix = [[0] * num_cities for _ in range(num_cities)]

    for i in range(1, num_cities + 1):
        for j in range(1, num_cities + 1):
            distance_matrix[i - 1][j - 1] = calculate_distance(coordinates[i], coordinates[j])

    return distance_matrix

# Example usage
file_path = "eil101.tsp"
coordinates = read_tsp_file(file_path)
distance_matrix = create_distance_matrix(coordinates)

In [ ]:
!pip install networkx --upgrade


# ***Tests pour comprendre le principe***

In [ ]:
import numpy as np
import random
import time
import networkx as nx
from networkx.algorithms.matching import min_weight_matching
from networkx.algorithms.tree.mst import minimum_spanning_tree

# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

def recherche_tabou_partiel(matrice_distances, solution_partielle):
    taille_liste_tabou = 5  # Définissez la taille de la liste tabou
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    meilleure_solution = solution_partielle[:]
    cout_meilleure_solution = calculer_cout(meilleure_solution, matrice_distances)
    liste_tabou = []

    voisinage = [solution_partielle + [ville] for ville in villes_restantes]
    couts_voisinage = [calculer_cout(voisin, matrice_distances) for voisin in voisinage]

    voisinage_et_couts = sorted([(voisin, cout) for voisin, cout in zip(voisinage, couts_voisinage)], key=lambda x: x[1])
    for voisin, cout_voisin in voisinage_et_couts:
        if (tuple(voisin), cout_voisin) not in liste_tabou or cout_voisin < cout_meilleure_solution:
            meilleure_solution = voisin
            cout_meilleure_solution = cout_voisin
            break

    if len(liste_tabou) >= taille_liste_tabou:
        liste_tabou.pop(0)
    liste_tabou.append((tuple(meilleure_solution), cout_meilleure_solution))

    return meilleure_solution, cout_meilleure_solution

def christofides_partiel(matrice_distances, solution_partielle):
    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    # Construire le sous-graphe avec des sommets de degré impair
    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    # Calculer un couplage parfait de poids minimum dans le sous-graphe
    matching = nx.algorithms.matching.max_weight_matching(odd_graph)

    # Ajouter les arêtes du couplage parfait au sous-arbre couvrant minimum
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    # Trouver un circuit eulérien dans le graphe obtenu
    eulerian_circuit = list(nx.eulerian_circuit(T))

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

class HyperHeuristicMMAS:
    def __init__(self, heuristics, num_ants, max_iter, alpha, beta, rho, q, tau_min, tau_max):
        self.heuristics = heuristics
        self.num_ants = num_ants
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.tau_min = tau_min
        self.tau_max = tau_max
        self.num_heuristics = len(heuristics)
        self.pheromone = np.full((self.num_heuristics,), self.tau_max)

    def run(self, matrice_distances):
        best_solution = None
        best_cost = float('inf')
        num_villes = len(matrice_distances)

        for _ in range(self.max_iter):
            solutions = []
            for _ in range(self.num_ants):
                solution_partielle = [random.randint(0, num_villes-1)]
                while len(solution_partielle) < num_villes:
                    probabilities = self.compute_probabilities()
                    selected_heuristic_index = np.random.choice(self.num_heuristics, p=probabilities)
                    heuristic = self.heuristics[selected_heuristic_index]
                    solution_partielle, _ = heuristic(matrice_distances, solution_partielle,)

                cout = calculer_cout(solution_partielle, matrice_distances)
                solutions.append((solution_partielle, cout))

            self.update_pheromone(solutions)
            best_ant_solution, best_ant_cost = min(solutions, key=lambda x: x[1])
            if best_ant_cost < best_cost:
                best_solution = best_ant_solution
                best_cost = best_ant_cost

        return best_solution, best_cost

    def compute_probabilities(self):
        probabilities = (self.pheromone ** self.alpha).astype(np.float64)
        probabilities /= np.sum(probabilities)
        return probabilities

    def update_pheromone(self, solutions):
        best_ant_solution, best_ant_cost = min(solutions, key=lambda x: x[1])
        delta_tau = self.q / best_ant_cost if best_ant_cost != 0 else self.q / 1e-6

        self.pheromone = (1 - self.rho) * self.pheromone
        for i in range(self.num_heuristics):
            if i in best_ant_solution:
                self.pheromone[i] += delta_tau

        self.pheromone = np.clip(self.pheromone, self.tau_min, self.tau_max)

# Exemple d'utilisation
heuristics = [recherche_tabou_partiel, christofides_partiel, nearest_neighbour_partiel]
num_ants = 5
max_iter = 5
alpha = 2
beta = 5
rho = 0.6
q = 60
tau_min = 0.1
tau_max = 10

hyper_heuristic = HyperHeuristicMMAS(heuristics, num_ants, max_iter, alpha, beta, rho, q, tau_min, tau_max)
best_solution, best_cost = hyper_heuristic.run(distances)

print("Meilleure solution trouvée:", best_solution)
print("Coût de la meilleure solution:", best_cost)


# ***MMAS HyperHeuristic : Approche pas complétement Constructive***

In [ ]:
import numpy as np
import random
import networkx as nx
from networkx.algorithms.matching import min_weight_matching
from networkx.algorithms.tree.mst import minimum_spanning_tree

# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

def christofides_partiel(matrice_distances, solution_partielle):
    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    matching = nx.algorithms.matching.max_weight_matching(odd_graph)
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    if nx.is_eulerian(T):
        eulerian_circuit = list(nx.eulerian_circuit(T))
    else:
        raise Exception("Le graphe résultant n'est pas eulérien")

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def loneliest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    loneliest_city = max(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(loneliest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def deux_opt(solution, matrice_distances):
    n = len(solution)
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if j - i == 1:
                    continue
                new_solution = solution[:i + 1] + solution[i + 1:j + 1][::-1] + solution[j + 1:]
                if calculer_cout(new_solution, matrice_distances) < calculer_cout(best, matrice_distances):
                    best = new_solution
                    improved = True
        solution = best
    return best

class HyperHeuristicMMAS:
    def __init__(self, heuristics, num_ants, max_iter, alpha, beta, rho, q, tau_min, tau_max):
        self.heuristics = heuristics
        self.num_ants = num_ants
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.tau_min = tau_min
        self.tau_max = tau_max
        self.num_heuristics = len(heuristics)
        self.pheromone = np.full((self.num_heuristics,), self.tau_max)

    def run(self, matrice_distances):
        best_solution = None
        best_cost = float('inf')
        num_villes = len(matrice_distances)

        for _ in range(self.max_iter):
            solutions = []
            for _ in range(self.num_ants):
                solution_partielle = [random.randint(0, num_villes-1)]
                while len(solution_partielle) < num_villes:
                    probabilities = self.compute_probabilities()
                    selected_heuristic_index = np.random.choice(self.num_heuristics, p=probabilities)
                    heuristic = self.heuristics[selected_heuristic_index]
                    solution_partielle, _ = heuristic(matrice_distances, solution_partielle)

                if len(solution_partielle) == num_villes:
                    solution_partielle = deux_opt(solution_partielle, matrice_distances)
                cout = calculer_cout(solution_partielle, matrice_distances)
                solutions.append((solution_partielle, cout))

            self.update_pheromone(solutions)
            best_ant_solution, best_ant_cost = min(solutions, key=lambda x: x[1])
            if best_ant_cost < best_cost:
                best_solution = best_ant_solution
                best_cost = best_ant_cost

        return best_solution, best_cost

    def compute_probabilities(self):
        probabilities = (self.pheromone ** self.alpha).astype(np.float64)
        probabilities /= np.sum(probabilities)
        return probabilities

    def update_pheromone(self, solutions):
        best_ant_solution, best_ant_cost = min(solutions, key=lambda x: x[1])
        delta_tau = self.q / best_ant_cost if best_ant_cost != 0 else self.q / 1e-6

        self.pheromone = (1 - self.rho) * self.pheromone
        for i in range(self.num_heuristics):
            if i in best_ant_solution:
                self.pheromone[i] += delta_tau

        self.pheromone = np.clip(self.pheromone, self.tau_min, self.tau_max)

# Exemple d'utilisation
nb_villes = 16  # Nombre de villes pour le test
distances = np.random.randint(1, 100, size=(nb_villes, nb_villes))
np.fill_diagonal(distances, 0)  # Les distances de chaque ville à elle-même sont 0

heuristics = [christofides_partiel, nearest_neighbour_partiel, loneliest_neighbour_partiel]
num_ants = 5
max_iter = 5
alpha = 2
beta = 5
rho = 0.6
q = 60
tau_min = 0.1
tau_max = 10
distances=[
[0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
[10, 0, 25, 35, 15, 20, 30, 5, 40, 35, 45, 50, 55, 60, 65, 70],
[15, 25, 0, 5, 10, 30, 40, 15, 25, 30, 20, 45, 30, 35, 40, 50],
[20, 35, 5, 0, 40, 25, 15, 10, 20, 45, 30, 35, 40, 50, 55, 60],
[25, 15, 10, 40, 0, 20, 10, 25, 30, 35, 50, 55, 60, 65, 70, 75],
[30, 20, 30, 25, 20, 0, 35, 30, 5, 15, 40, 45, 50, 55, 60, 65],
[35, 30, 40, 15, 10, 35, 0, 20, 25, 30, 45, 60, 65, 70, 75, 80],
[40, 5, 15, 10, 25, 30, 20, 0, 35, 40, 55, 35, 40, 45, 50, 55],
[45, 40, 25, 20, 30, 5, 25, 35, 0, 15, 10, 25, 30, 35, 40, 45],
[50, 35, 30, 45, 35, 15, 30, 40, 15, 0, 25, 20, 15, 20, 25, 30],
[55, 45, 20, 30, 50, 40, 45, 55, 10, 25, 0, 15, 30, 35, 40, 45],
[60, 50, 45, 35, 55, 45, 60, 35, 25, 20, 15, 0, 10, 15, 20, 25],
[65, 55, 30, 40, 60, 50, 65, 40, 30, 15, 30, 10, 0, 25, 30, 35],
[70, 60, 35, 50, 65, 55, 70, 45, 35, 20, 35, 15, 25, 0, 10, 15],
[75, 65, 40, 55, 70, 60, 75, 50, 40, 25, 40, 20, 30, 10, 0, 5],
[80, 70, 50, 60, 75, 65, 80, 55, 45, 30, 45, 25, 35, 15, 5, 0]
]

hyper_heuristic = HyperHeuristicMMAS(heuristics, num_ants, max_iter, alpha, beta, rho, q, tau_min, tau_max)
best_solution, best_cost = hyper_heuristic.run(distances)

print("Meilleure solution trouvée:", best_solution)
print("Coût de la meilleure solution:", best_cost)


Meilleure solution trouvée: [10, 8, 5, 4, 6, 3, 7, 1, 0, 2, 12, 9, 15, 14, 13, 11]
Coût de la meilleure solution: 220


# ***MMAS HyperHeuristic : Approche Constructive qui marche à merveille***

High Level : Min-Max Ant System

Low Level : 3 Heuristiques

In [ ]:
import numpy as np
import random
import networkx as nx
from networkx.algorithms.matching import min_weight_matching
from networkx.algorithms.tree.mst import minimum_spanning_tree

# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

def christofides_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    matching = nx.algorithms.matching.max_weight_matching(odd_graph)
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    if nx.is_eulerian(T):
        eulerian_circuit = list(nx.eulerian_circuit(T))
    else:
        raise Exception("Le graphe résultant n'est pas eulérien")

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def loneliest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    loneliest_city = max(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(loneliest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

class HyperHeuristicMMAS:
    def __init__(self, heuristics, num_ants, max_iter, alpha, beta, rho, q, tau_min, tau_max):
        self.heuristics = heuristics
        self.num_ants = num_ants
        self.max_iter = max_iter
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.tau_min = tau_min
        self.tau_max = tau_max
        self.num_heuristics = len(heuristics)
        self.pheromone = np.full((self.num_heuristics,), self.tau_max)

    def run(self, matrice_distances):
        best_solution = None
        best_cost = float('inf')
        num_villes = len(matrice_distances)

        for iteration in range(self.max_iter):
            solutions = []
            for ant in range(self.num_ants):
                solution_partielle = [random.randint(0, num_villes-1)]
                while len(solution_partielle) < num_villes:
                    probabilities = self.compute_probabilities()
                    selected_heuristic_index = np.random.choice(self.num_heuristics, p=probabilities)
                    heuristic = self.heuristics[selected_heuristic_index]
                    solution_partielle, _ = heuristic(matrice_distances, solution_partielle)

                #if len(solution_partielle) == num_villes:
                    #solution_partielle = deux_opt(solution_partielle, matrice_distances)
                cout = calculer_cout(solution_partielle, matrice_distances)
                solutions.append((solution_partielle, cout))

            self.update_pheromone(solutions)
            best_ant_solution, best_ant_cost = min(solutions, key=lambda x: x[1])
            if best_ant_cost < best_cost:
                best_solution = best_ant_solution
                best_cost = best_ant_cost

        return best_solution, best_cost

    def compute_probabilities(self):
        probabilities = (self.pheromone ** self.alpha).astype(np.float64)
        probabilities /= np.sum(probabilities)
        return probabilities

    def update_pheromone(self, solutions):
        best_ant_solution, best_ant_cost = min(solutions, key=lambda x: x[1])
        delta_tau = self.q / best_ant_cost if best_ant_cost != 0 else self.q / 1e-6

        self.pheromone = (1 - self.rho) * self.pheromone
        for i in range(self.num_heuristics):
            if i in best_ant_solution:
                self.pheromone[i] += delta_tau

        self.pheromone = np.clip(self.pheromone, self.tau_min, self.tau_max)

# Exemple d'utilisation
nb_villes = 16  # Nombre de villes pour le test
distances = np.random.randint(1, 100, size=(nb_villes, nb_villes))
np.fill_diagonal(distances, 0)  # Les distances de chaque ville à elle-même sont 0

heuristics = [christofides_partiel, nearest_neighbour_partiel, loneliest_neighbour_partiel]
num_ants = 5
max_iter = 1
alpha = 2
beta = 5
rho = 0.6
q = 60
tau_min = 0.1
tau_max = 10
distances=[
[0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
[10, 0, 25, 35, 15, 20, 30, 5, 40, 35, 45, 50, 55, 60, 65, 70],
[15, 25, 0, 5, 10, 30, 40, 15, 25, 30, 20, 45, 30, 35, 40, 50],
[20, 35, 5, 0, 40, 25, 15, 10, 20, 45, 30, 35, 40, 50, 55, 60],
[25, 15, 10, 40, 0, 20, 10, 25, 30, 35, 50, 55, 60, 65, 70, 75],
[30, 20, 30, 25, 20, 0, 35, 30, 5, 15, 40, 45, 50, 55, 60, 65],
[35, 30, 40, 15, 10, 35, 0, 20, 25, 30, 45, 60, 65, 70, 75, 80],
[40, 5, 15, 10, 25, 30, 20, 0, 35, 40, 55, 35, 40, 45, 50, 55],
[45, 40, 25, 20, 30, 5, 25, 35, 0, 15, 10, 25, 30, 35, 40, 45],
[50, 35, 30, 45, 35, 15, 30, 40, 15, 0, 25, 20, 15, 20, 25, 30],
[55, 45, 20, 30, 50, 40, 45, 55, 10, 25, 0, 15, 30, 35, 40, 45],
[60, 50, 45, 35, 55, 45, 60, 35, 25, 20, 15, 0, 10, 15, 20, 25],
[65, 55, 30, 40, 60, 50, 65, 40, 30, 15, 30, 10, 0, 25, 30, 35],
[70, 60, 35, 50, 65, 55, 70, 45, 35, 20, 35, 15, 25, 0, 10, 15],
[75, 65, 40, 55, 70, 60, 75, 50, 40, 25, 40, 20, 30, 10, 0, 5],
[80, 70, 50, 60, 75, 65, 80, 55, 45, 30, 45, 25, 35, 15, 5, 0]
]

hyper_heuristic = HyperHeuristicMMAS(heuristics, num_ants, max_iter, alpha, beta, rho, q, tau_min, tau_max)
best_solution, best_cost = hyper_heuristic.run(distances)

print("Meilleure solution trouvée:", best_solution)
print("Coût de la meilleure solution:", best_cost)

Meilleure solution trouvée: [1, 0, 10, 8, 5, 9, 3, 2, 4, 15, 6, 7, 11, 12, 13, 14]
Coût de la meilleure solution: 475


# ***Khouroudj 3an saytara***

In [ ]:
import numpy as np
import random
import networkx as nx
from networkx.algorithms.matching import min_weight_matching
from networkx.algorithms.tree.mst import minimum_spanning_tree

# Fonction pour calculer le coût d'une solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance entre la ville i et i+1
    return cout

def christofides_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    matching = min_weight_matching(odd_graph, maxcardinality=True)
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    if nx.is_eulerian(T):
        eulerian_circuit = list(nx.eulerian_circuit(T))
    else:
        raise Exception("Le graphe résultant n'est pas eulérien")

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def loneliest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    loneliest_city = max(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(loneliest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def deux_opt(solution, matrice_distances):
    n = len(solution)
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if j - i == 1:
                    continue
                new_solution = solution[:i + 1] + solution[i + 1:j + 1][::-1] + solution[j + 1:]
                if calculer_cout(new_solution, matrice_distances) < calculer_cout(best, matrice_distances):
                    best = new_solution
                    improved = True
        solution = best
    return best

class HyperHeuristicLearningAutomata:
    def __init__(self, heuristics, num_ants, alpha, beta, rho, q, reward, penalty):
        self.heuristics = heuristics
        self.num_ants = num_ants
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.reward = reward
        self.penalty = penalty
        self.num_heuristics = len(heuristics)
        self.probabilities = np.full((self.num_heuristics,), 1.0 / self.num_heuristics)

    def run(self, matrice_distances):
        best_solution = None
        best_cost = float('inf')
        num_villes = len(matrice_distances)

        solutions = []
        for ant in range(self.num_ants):
            solution_partielle = [random.randint(0, num_villes-1)]
            while len(solution_partielle) < num_villes:
                selected_heuristic_index = np.random.choice(self.num_heuristics, p=self.probabilities)
                heuristic = self.heuristics[selected_heuristic_index]
                solution_partielle, _ = heuristic(matrice_distances, solution_partielle)

            if len(solution_partielle) == num_villes:
                solution_partielle = deux_opt(solution_partielle, matrice_distances)
            cout = calculer_cout(solution_partielle, matrice_distances)
            solutions.append((solution_partielle, cout))

            # Mise à jour des probabilités des heuristiques
            if cout < best_cost:
                self.probabilities[selected_heuristic_index] *= (1 + self.reward)
                best_cost = cout
                best_solution = solution_partielle
            else:
                self.probabilities[selected_heuristic_index] *= (1 - self.penalty)

        # Normalisation des probabilités
        self.probabilities /= np.sum(self.probabilities)

        return best_solution, best_cost

# Exemple d'utilisation
nb_villes = 16  # Nombre de villes pour le test
distances = np.random.randint(1, 100, size=(nb_villes, nb_villes))
np.fill_diagonal(distances, 0)  # Les distances de chaque ville à elle-même sont 0

heuristics = [christofides_partiel, nearest_neighbour_partiel, loneliest_neighbour_partiel]
num_ants = 5
alpha = 2
beta = 5
rho = 0.6
q = 60
reward = 0.1
penalty = 0.1

hyper_heuristic = HyperHeuristicLearningAutomata(heuristics, num_ants, alpha, beta, rho, q, reward, penalty)
best_solution, best_cost = hyper_heuristic.run(distances)

print("Meilleure solution trouvée:", best_solution)
print("Coût de la meilleure solution:", best_cost)


In [ ]:
import numpy as np
import random
import networkx as nx
from networkx.algorithms.matching import min_weight_matching
from networkx.algorithms.tree.mst import minimum_spanning_tree

# Function to calculate the cost of a solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance between city i and i+1
    return cout

# Heuristic functions to add one more city to the partial solution
def christofides_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    matching = nx.algorithms.matching.max_weight_matching(odd_graph)
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    if nx.is_eulerian(T):
        eulerian_circuit = list(nx.eulerian_circuit(T))
    else:
        raise Exception("The resulting graph is not Eulerian")

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def loneliest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    loneliest_city = max(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(loneliest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def deux_opt(solution, matrice_distances):
    n = len(solution)
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if j - i == 1:
                    continue
                new_solution = solution[:i + 1] + solution[i + 1:j + 1][::-1] + solution[j + 1:]
                if calculer_cout(new_solution, matrice_distances) < calculer_cout(best, matrice_distances):
                    best = new_solution
                    improved = True
        solution = best
    return best

class HyperHeuristicConstructive:
    def __init__(self, heuristics, num_ants, reward, penalty):
        self.heuristics = heuristics
        self.num_ants = num_ants
        self.reward = reward
        self.penalty = penalty
        self.num_heuristics = len(heuristics)
        self.probabilities = np.full((self.num_heuristics,), 1.0 / self.num_heuristics)

    def run(self, matrice_distances):
        best_solution = None
        best_cost = float('inf')
        num_villes = len(matrice_distances)

        for ant in range(self.num_ants):
            solution_partielle = [random.randint(0, num_villes-1)]
            while len(solution_partielle) < num_villes:
                propositions = []
                for h_index in range(self.num_heuristics):
                    heuristic = self.heuristics[h_index]
                    proposition, _ = heuristic(matrice_distances, solution_partielle.copy())
                    propositions.append((proposition, h_index))

                # Select the best proposition
                best_proposition = min(propositions, key=lambda x: calculer_cout(x[0], matrice_distances))
                solution_partielle = best_proposition[0]
                best_heuristic_index = best_proposition[1]

                # Reward and penalize heuristics
                for h_index, (_, _) in enumerate(propositions):
                    if h_index == best_heuristic_index:
                        self.probabilities[h_index] *= (1 + self.reward)
                    else:
                        self.probabilities[h_index] *= (1 - self.penalty)

                # Normalize probabilities
                self.probabilities /= np.sum(self.probabilities)

            #if len(solution_partielle) == num_villes:
               # solution_partielle = deux_opt(solution_partielle, matrice_distances)
            cout = calculer_cout(solution_partielle, matrice_distances)

            if cout < best_cost:
                best_cost = cout
                best_solution = solution_partielle

        return best_solution, best_cost

# Example usage
nb_villes = 101
heuristics = [christofides_partiel, nearest_neighbour_partiel, loneliest_neighbour_partiel]
num_ants = 5
reward = 0.1
penalty = 0.1

hyper_heuristic = HyperHeuristicConstructive(heuristics, num_ants, reward, penalty)
best_solution, best_cost = hyper_heuristic.run(distances)

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)


Best solution found: [86, 96, 94, 93, 5, 95, 98, 58, 91, 36, 97, 84, 92, 90, 99, 43, 13, 41, 56, 1, 72, 20, 71, 73, 21, 74, 55, 38, 22, 66, 24, 54, 53, 79, 67, 76, 2, 78, 32, 0, 68, 26, 100, 52, 57, 39, 25, 11, 75, 49, 80, 8, 50, 19, 29, 69, 30, 87, 6, 81, 47, 46, 35, 48, 18, 10, 61, 9, 89, 31, 62, 63, 45, 7, 44, 16, 83, 4, 59, 82, 17, 51, 88, 12, 27, 3, 23, 28, 77, 33, 34, 70, 65, 64, 40, 14, 42, 37, 85, 15, 60]
Cost of the best solution: 803.795813423334


In [ ]:
import xml.etree.ElementTree as ET
import numpy as np

def load_xml_and_extract_matrix(xml_file_path):
    # Charger le fichier XML
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # Trouver l'élément 'graph' qui contient les données de la matrice
    graph_element = root.find('graph')

    # Extraire la matrice symétrique
    return extract_symmetric_matrix(graph_element)

def extract_symmetric_matrix(graph_element):
    num_vertices = len(graph_element)  # Nombre de vertices (supposé carré)
    matrix = np.zeros((num_vertices, num_vertices), dtype=float)  # Initialiser la matrice de zéros

    # Parcourir chaque vertex
    for i, vertex in enumerate(graph_element):
        # Parcourir chaque edge du vertex
        for edge in vertex:
            j = int(edge.text) - 1  # Index du vertex connecté (base 1 à base 0)
            if i != j:  # Éviter de remplir la diagonale
                cost = float(edge.attrib['cost'])  # Coût de l'edge
                matrix[i, j] = cost
                matrix[j, i] = cost  # Assurer la symétrie

    return matrix

# Utiliser la fonction
file_path = '/content/gr48.xml'  # Remplacer par le chemin réel du fichier
distance_matrix= load_xml_and_extract_matrix(file_path)

# ***She is good***

In [ ]:
import numpy as np
import random
import networkx as nx
from networkx.algorithms.matching import min_weight_matching
from networkx.algorithms.tree.mst import minimum_spanning_tree
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to calculate the cost of a solution
def calculer_cout(S, matrice_distances):
    cout = 0
    n = len(S)
    for i in range(n):
        cout += matrice_distances[S[i]][S[(i + 1) % n]]  # Distance between city i and i+1
    return cout

# Heuristic functions to add one more city to the partial solution
def christofides_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    G = nx.complete_graph(len(matrice_distances))
    for u, v in G.edges():
        G[u][v]['weight'] = matrice_distances[u][v]

    T = minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)

    matching = nx.algorithms.matching.max_weight_matching(odd_graph)
    for u, v in matching:
        T.add_edge(u, v, weight=matrice_distances[u][v])

    if nx.is_eulerian(T):
        eulerian_circuit = list(nx.eulerian_circuit(T))
    else:
        raise Exception("The resulting graph is not Eulerian")

    visited = set(solution_partielle)
    next_city = None
    for u, v in eulerian_circuit:
        if u not in visited:
            next_city = u
            break
        if v not in visited:
            next_city = v
            break

    if next_city is not None:
        solution_partielle.append(next_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def nearest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    nearest_city = min(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(nearest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def loneliest_neighbour_partiel(matrice_distances, solution_partielle):
    nb_villes = len(matrice_distances)
    villes_restantes = list(set(range(nb_villes)) - set(solution_partielle))

    if not villes_restantes:
        return solution_partielle, calculer_cout(solution_partielle, matrice_distances)

    dernier = solution_partielle[-1]
    loneliest_city = max(villes_restantes, key=lambda x: matrice_distances[dernier][x])
    solution_partielle.append(loneliest_city)

    cout_actuel = calculer_cout(solution_partielle, matrice_distances)

    return solution_partielle, cout_actuel

def deux_opt(solution, matrice_distances):
    n = len(solution)
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(n - 1):
            for j in range(i + 2, n):
                if j - i == 1:
                    continue
                new_solution = solution[:i + 1] + solution[i + 1:j + 1][::-1] + solution[j + 1:]
                if calculer_cout(new_solution, matrice_distances) < calculer_cout(best, matrice_distances):
                    best = new_solution
                    improved = True
        solution = best
    return best

class HyperHeuristicConstructive:
    def __init__(self, heuristics, num_ants, reward, penalty):
        self.heuristics = heuristics
        self.num_ants = num_ants
        self.reward = reward
        self.penalty = penalty
        self.num_heuristics = len(heuristics)
        self.probabilities = np.full((self.num_heuristics,), 1.0 / self.num_heuristics)

    def run(self, matrice_distances):
        best_solution = None
        best_cost = float('inf')
        num_villes = len(matrice_distances)

        with ThreadPoolExecutor(max_workers=self.num_ants) as executor:
            futures = [executor.submit(self.construct_solution, matrice_distances) for _ in range(self.num_ants)]
            for future in as_completed(futures):
                solution_partielle, cout = future.result()
                if cout < best_cost:
                    best_cost = cout
                    best_solution = solution_partielle

        return best_solution, best_cost

    def construct_solution(self, matrice_distances):
        num_villes = len(matrice_distances)
        solution_partielle = [random.randint(0, num_villes-1)]
        while len(solution_partielle) < num_villes:
            propositions = []
            for h_index in range(self.num_heuristics):
                heuristic = self.heuristics[h_index]
                proposition, _ = heuristic(matrice_distances, solution_partielle.copy())
                propositions.append((proposition, h_index))

            # Select the best proposition
            best_proposition = min(propositions, key=lambda x: calculer_cout(x[0], matrice_distances))
            solution_partielle = best_proposition[0]
            best_heuristic_index = best_proposition[1]

            # Reward and penalize heuristics
            for h_index, (_, _) in enumerate(propositions):
                if h_index == best_heuristic_index:
                    self.probabilities[h_index] *= (1 + self.reward)
                else:
                    self.probabilities[h_index] *= (1 - self.penalty)

            # Normalize probabilities
            self.probabilities /= np.sum(self.probabilities)

        if len(solution_partielle) == num_villes:
            solution_partielle = deux_opt(solution_partielle, matrice_distances)
        cout = calculer_cout(solution_partielle, matrice_distances)

        return solution_partielle, cout

# Example usage
nb_villes = 48

heuristics = [christofides_partiel, nearest_neighbour_partiel, loneliest_neighbour_partiel]
num_ants = 5
reward = 0.1
penalty = 0.1
distances = distance_matrix
hyper_heuristic = HyperHeuristicConstructive(heuristics, num_ants, reward, penalty)
best_solution, best_cost = hyper_heuristic.run(distances)

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)


Best solution found: [25, 31, 19, 26, 15, 11, 47, 14, 28, 37, 39, 43, 42, 45, 16, 27, 5, 12, 9, 35, 4, 10, 30, 8, 6, 32, 40, 21, 33, 23, 22, 36, 44, 41, 46, 38, 0, 34, 18, 2, 29, 17, 24, 1, 3, 20, 7, 13]
Cost of the best solution: 5112.0


# ***Exploitation de la logique tabou comme stratégie de haut niveau***

In [ ]:
import numpy as np
import random
import copy

# Heuristique de bas niveau : 2-opt
def two_opt(route, distance_matrix):
    best = route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route)):
                if j - i == 1: continue
                new_route = route[:]
                new_route[i:j] = route[j - 1:i - 1:-1]
                if calculate_total_distance(new_route, distance_matrix) < calculate_total_distance(best, distance_matrix):
                    best = new_route
                    improved = True
        route = best
    return best

# Fonction pour calculer la distance totale d'une route donnée
def calculate_total_distance(route, distance_matrix):
    total_distance = 0
    for i in range(len(route)):
        total_distance += distance_matrix[route[i-1]][route[i]]
    return total_distance

# Recherche tabou
def tabu_search(distance_matrix, initial_solution, num_iterations, tabu_tenure):
    best_solution = initial_solution
    best_cost = calculate_total_distance(initial_solution, distance_matrix)
    current_solution = initial_solution
    tabu_list = []

    for _ in range(num_iterations):
        neighborhood = generate_neighborhood(current_solution, distance_matrix)
        best_candidate = None
        best_candidate_cost = float('inf')

        for candidate in neighborhood:
            if candidate in tabu_list:
                continue
            candidate_cost = calculate_total_distance(candidate, distance_matrix)
            if candidate_cost < best_candidate_cost:
                best_candidate = candidate
                best_candidate_cost = candidate_cost

        if best_candidate is None:
            break

        current_solution = best_candidate
        tabu_list.append(best_candidate)

        if len(tabu_list) > tabu_tenure:
            tabu_list.pop(0)

        if best_candidate_cost < best_cost:
            best_solution = best_candidate
            best_cost = best_candidate_cost

    return best_solution, best_cost

# Générer le voisinage par 2-opt
def generate_neighborhood(route, distance_matrix):
    neighborhood = []
    for i in range(len(route) - 1):
        for j in range(i + 1, len(route)):
            if i == 0 and j == len(route) - 1:
                continue
            new_route = route[:]
            new_route[i:j + 1] = reversed(route[i:j + 1])
            neighborhood.append(new_route)
    return neighborhood

# Exemple d'application
if __name__ == "__main__":
    # Matrice de distance (exemple avec 5 villes)
    distance_matrix = np.array(distances)

    # Solution initiale aléatoire
    initial_solution = list(range(len(distance_matrix)))
    random.shuffle(initial_solution)

    # Paramètres de la recherche tabou
    num_iterations = 100
    tabu_tenure = 5

    # Exécution de la recherche tabou
    best_solution, best_cost = tabu_search(distance_matrix, initial_solution, num_iterations, tabu_tenure)

    print("Meilleure solution trouvée:", best_solution)
    print("Coût de la meilleure solution:", best_cost)


Meilleure solution trouvée: [99, 41, 56, 14, 42, 13, 43, 37, 85, 15, 60, 83, 4, 59, 82, 16, 44, 7, 45, 46, 35, 48, 63, 62, 89, 31, 9, 29, 50, 19, 65, 64, 70, 34, 33, 8, 80, 32, 77, 78, 2, 76, 28, 23, 54, 24, 38, 66, 22, 74, 40, 21, 73, 55, 3, 53, 79, 67, 11, 25, 27, 75, 49, 0, 68, 69, 30, 87, 61, 10, 18, 47, 81, 6, 17, 88, 51, 26, 100, 52, 39, 20, 71, 72, 1, 86, 96, 12, 57, 5, 93, 94, 91, 58, 95, 98, 92, 84, 90, 97, 36]
Coût de la meilleure solution: 705.6378927444151


In [ ]:
import random
import networkx as nx

# Define the distance matrix
'''distance_matrix = [
    [0, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80],
    [10, 0, 25, 35, 15, 20, 30, 5, 40, 35, 45, 50, 55, 60, 65, 70],
    [15, 25, 0, 5, 10, 30, 40, 15, 25, 30, 20, 45, 30, 35, 40, 50],
    [20, 35, 5, 0, 40, 25, 15, 10, 20, 45, 30, 35, 40, 50, 55, 60],
    [25, 15, 10, 40, 0, 20, 10, 25, 30, 35, 50, 55, 60, 65, 70, 75],
    [30, 20, 30, 25, 20, 0, 35, 30, 5, 15, 40, 45, 50, 55, 60, 65],
    [35, 30, 40, 15, 10, 35, 0, 20, 25, 30, 45, 60, 65, 70, 75, 80],
    [40, 5, 15, 10, 25, 30, 20, 0, 35, 40, 55, 35, 40, 45, 50, 55],
    [45, 40, 25, 20, 30, 5, 25, 35, 0, 15, 10, 25, 30, 35, 40, 45],
    [50, 35, 30, 45, 35, 15, 30, 40, 15, 0, 25, 20, 15, 20, 25, 30],
    [55, 45, 20, 30, 50, 40, 45, 55, 10, 25, 0, 15, 30, 35, 40, 45],
    [60, 50, 45, 35, 55, 45, 60, 35, 25, 20, 15, 0, 10, 15, 20, 25],
    [65, 55, 30, 40, 60, 50, 65, 40, 30, 15, 30, 10, 0, 25, 30, 35],
    [70, 60, 35, 50, 65, 55, 70, 45, 35, 20, 35, 15, 25, 0, 10, 15],
    [75, 65, 40, 55, 70, 60, 75, 50, 40, 25, 40, 20, 30, 10, 0, 5],
    [80, 70, 50, 60, 75, 65, 80, 55, 45, 30, 45, 25, 35, 15, 5, 0]
]'''

def distance(city1, city2):
    return distance_matrix[city1][city2]

def evaluate_solution(solution):
    return sum(distance(solution[i], solution[i + 1]) for i in range(len(solution) - 1)) + distance(solution[-1], solution[0])

def nearest_neighbor(cities):
    unvisited = cities[:]
    solution = [unvisited.pop(0)]
    while unvisited:
        nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
        solution.append(nearest)
        unvisited.remove(nearest)
    return solution

def christofides_tsp(distance_matrix):
    G = nx.Graph()
    num_nodes = len(distance_matrix)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            G.add_edge(i, j, weight=distance_matrix[i][j])

    mst = nx.minimum_spanning_tree(G)

    odd_degree_nodes = [v for v, degree in mst.degree() if degree % 2 == 1]

    odd_subgraph = G.subgraph(odd_degree_nodes)
    matching = nx.algorithms.matching.max_weight_matching(odd_subgraph, maxcardinality=True, weight='weight')

    multigraph = nx.MultiGraph(mst)
    for u, v in matching:
        multigraph.add_edge(u, v, weight=G[u][v]['weight'])

    eulerian_circuit = list(nx.eulerian_circuit(multigraph))

    visited = set()
    tsp_path = []
    for u, v in eulerian_circuit:
        if u not in visited:
            tsp_path.append(u)
            visited.add(u)
    tsp_path.append(tsp_path[0])

    return tsp_path

def two_opt(solution):
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(1, len(solution) - 1):
            for j in range(i + 1, len(solution)):
                if j - i == 1: continue
                new_solution = solution[:i] + solution[i:j][::-1] + solution[j:]
                if evaluate_solution(new_solution) < evaluate_solution(best):
                    best = new_solution
                    improved = True
    return best

def insertion_heuristic(cities):
    solution = [cities.pop(0)]
    while cities:
        best_increase = float('inf')
        best_city = None
        best_position = None
        for city in cities:
            for i in range(len(solution) + 1):
                new_solution = solution[:i] + [city] + solution[i:]
                increase = evaluate_solution(new_solution) - evaluate_solution(solution)
                if increase < best_increase:
                    best_increase = increase
                    best_city = city
                    best_position = i
        solution.insert(best_position, best_city)
        cities.remove(best_city)
    return solution

def improved_nearest_neighbor(cities):
    best_solution = None
    best_distance = float('inf')
    for start in range(len(cities)):
        unvisited = cities[:]
        solution = [unvisited.pop(start)]
        while unvisited:
            nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
            solution.append(nearest)
            unvisited.remove(nearest)
        total_distance = evaluate_solution(solution)
        if total_distance < best_distance:
            best_solution = solution
            best_distance = total_distance
    return best_solution

heuristics = {
    "nearest_neighbor": nearest_neighbor,
    "christofides_tsp": lambda cities: christofides_tsp(distance_matrix),
    "two_opt": two_opt,
    "insertion_heuristic": insertion_heuristic,
    "improved_nearest_neighbor": improved_nearest_neighbor
}

scores = {name: 1 for name in heuristics}
uses = {name: 0 for name in heuristics}

used_heuristics = []

def select_heuristic(scores):
    global used_heuristics
    if len(used_heuristics) == len(heuristics):
        used_heuristics = []

    remaining_heuristics = [h for h in heuristics if h not in used_heuristics]
    selected_heuristic = random.choice(remaining_heuristics)
    used_heuristics.append(selected_heuristic)
    return selected_heuristic

def update_scores(selected_heuristic, improvement):
    if improvement:
        scores[selected_heuristic] += 1
    else:
        scores[selected_heuristic] = max(1, scores[selected_heuristic] - 1)

def tabu_search(initial_solution, max_iterations=40, tabu_size=7):
    current_solution = initial_solution
    best_solution = initial_solution
    tabu_list = []
    best_distances = []

    for iteration in range(max_iterations):
        selected_heuristic = select_heuristic(scores)
        new_solution = heuristics[selected_heuristic](current_solution.copy())

        if new_solution not in tabu_list:
            uses[selected_heuristic] += 1
            current_solution = new_solution
            if evaluate_solution(new_solution) < evaluate_solution(best_solution):
                best_solution = new_solution
                update_scores(selected_heuristic, True)
            else:
                update_scores(selected_heuristic, False)

            tabu_list.append(new_solution)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)

        best_distances.append(evaluate_solution(best_solution))

    return best_solution, best_distances

def generate_initial_solution(cities):
    return nearest_neighbor(cities)

# Test the implementation with the provided distance matrix
cities = list(range(len(distance_matrix)))
initial_solution = generate_initial_solution(cities.copy())
best_solution , ds = tabu_search(initial_solution)
print("Best solution found:", best_solution)
print("Total distance:", evaluate_solution(best_solution))


Best solution found: [49, 32, 80, 50, 8, 70, 64, 34, 33, 77, 78, 2, 76, 75, 67, 79, 28, 23, 24, 54, 38, 66, 22, 55, 74, 40, 21, 73, 72, 20, 71, 3, 53, 11, 25, 27, 52, 39, 57, 12, 1, 56, 14, 42, 41, 86, 96, 91, 58, 36, 97, 99, 90, 13, 43, 37, 85, 15, 60, 84, 92, 4, 83, 16, 44, 7, 45, 35, 48, 46, 18, 47, 61, 9, 10, 63, 62, 89, 31, 65, 19, 29, 69, 30, 87, 6, 81, 82, 17, 59, 98, 95, 94, 93, 5, 88, 51, 100, 26, 68, 0, 0]
Total distance: 702.9603861226374


In [ ]:
import random
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import time
from networkx.algorithms.matching import min_weight_matching

# Define the distance matrix


def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G

def distance(city1, city2):
    return distance_matrix[city1][city2]

def evaluate_solution(solution):
    return sum(distance(solution[i], solution[i + 1]) for i in range(len(solution) - 1)) + distance(solution[-1], solution[0])

def nearest_neighbor(cities):
    unvisited = cities[:]
    solution = [unvisited.pop(0)]
    while unvisited:
        nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
        solution.append(nearest)
        unvisited.remove(nearest)
    return solution

def christofides_tsp(distance_matrix):
    G = build_graph_from_distance_matrix(distance_matrix)
    return christofides_algorithm(G)

def two_opt(solution):
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(1, len(solution) - 1):
            for j in range(i + 1, len(solution)):
                if j - i == 1: continue
                new_solution = solution[:i] + solution[i:j][::-1] + solution[j:]
                if evaluate_solution(new_solution) < evaluate_solution(best):
                    best = new_solution
                    improved = True
    return best

def insertion_heuristic(cities):
    solution = [cities.pop(0)]
    while cities:
        best_increase = float('inf')
        best_city = None
        best_position = None
        for city in cities:
            for i in range(len(solution) + 1):
                new_solution = solution[:i] + [city] + solution[i:]
                increase = evaluate_solution(new_solution) - evaluate_solution(solution)
                if increase < best_increase:
                    best_increase = increase
                    best_city = city
                    best_position = i
        solution.insert(best_position, best_city)
        cities.remove(best_city)
    return solution

def improved_nearest_neighbor(cities):
    best_solution = None
    best_distance = float('inf')
    for start in range(len(cities)):
        unvisited = cities[:]
        solution = [unvisited.pop(start)]
        while unvisited:
            nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
            solution.append(nearest)
            unvisited.remove(nearest)
        total_distance = evaluate_solution(solution)
        if total_distance < best_distance:
            best_solution = solution
            best_distance = total_distance
    return best_solution

heuristics = {
    "nearest_neighbor": nearest_neighbor,
    "christofides_tsp": lambda cities: christofides_tsp(distance_matrix),
    "two_opt": two_opt,
    "insertion_heuristic": insertion_heuristic,
    "improved_nearest_neighbor": improved_nearest_neighbor
}

scores = {name: 1 for name in heuristics}
uses = {name: 0 for name in heuristics}
used_heuristics = []

def select_heuristic(scores):
    global used_heuristics
    if len(used_heuristics) == len(heuristics):
        used_heuristics = []

    remaining_heuristics = [h for h in heuristics if h not in used_heuristics]
    selected_heuristic = random.choice(remaining_heuristics)
    used_heuristics.append(selected_heuristic)
    return selected_heuristic

def update_scores(selected_heuristic, improvement):
    if improvement:
        scores[selected_heuristic] += 1
    else:
        scores[selected_heuristic] = max(1, scores[selected_heuristic] - 1)

def generer_solution_initiale(nb_villes):
    return random.sample(range(nb_villes), nb_villes)

def generer_voisinage(S):
    voisinage = []
    for i in range(1, len(S)):
        for j in range(i, len(S)):
            voisin = S[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

def recherche_locale_2opt(adjacency_matrix, route):
    route = [int(r) for r in route]
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route) - 1):
                current_distance = adjacency_matrix[route[i]][route[i + 1]] + \
                                   adjacency_matrix[route[j]][route[j + 1]]
                new_distance = adjacency_matrix[route[i]][route[j]] + \
                               adjacency_matrix[route[i + 1]][route[j + 1]]
                if new_distance < current_distance:
                    route[i + 1:j + 1] = route[i + 1:j + 1][::-1]
                    improved = True
    return route

def christofides_algorithm(G):
    T = nx.minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)
    matching = min_weight_matching(odd_graph)
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']
    return hamiltonian_circuit, total_cost

def calculer_cout(route, matrice_distances):
    cout_total = 0
    for i in range(len(route) - 1):
        distance = matrice_distances[route[i]][route[i + 1]]
        cout_total += distance
    cout_total += matrice_distances[route[-1]][route[0]]
    return cout_total

def recherche_tabou(G, nb_villes, nb_iterations_max, taille_liste_tabou, matrice_distances, diversifier, intensifier, stop_stagnation, max_iter_sans_amelioration, initialize_randomly, seuil_variance):
    if initialize_randomly:
        S = generer_solution_initiale(nb_villes)
        meilleur_cout = calculer_cout(S, matrice_distances)
    else:
        S, meilleur_cout = christofides_algorithm(G)
        S = recherche_locale_2opt(adjacency_matrix=matrice_distances, route=S)

    meilleure_solution = S[:]
    liste_tabou = []
    iter_sans_amelioration = 0
    var_couts = []

    for iteration in range(nb_iterations_max):
        selected_heuristic = select_heuristic(scores)
        S_prime = heuristics[selected_heuristic](S)

        if isinstance(S_prime[0], list):  # This check ensures S_prime is a list of integers
            S_prime = S_prime[0]

        cout_S_prime = calculer_cout(S_prime, matrice_distances)

        if (cout_S_prime < meilleur_cout and (S_prime not in liste_tabou or cout_S_prime < meilleur_cout)):
            meilleure_solution = S_prime[:]
            meilleur_cout = cout_S_prime
            update_scores(selected_heuristic, True)
            iter_sans_amelioration = 0
            var_couts.append(cout_S_prime)
        else:
            update_scores(selected_heuristic, False)
            iter_sans_amelioration += 1

        S = S_prime
        liste_tabou.append(S)
        if len(liste_tabou) > taille_liste_tabou:
            liste_tabou.pop(0)

        if iteration % 100 == 0 and iteration != 0:
            if random.random() > 0.5:
                if diversifier:
                    S = generer_solution_initiale(nb_villes)
            else:
                if intensifier:
                    if random.random() > 0.5:
                        S = recherche_locale_2opt(adjacency_matrix=matrice_distances, route=S)
                    else:
                        S = generer_solution_initiale(nb_villes)
                        liste_tabou = []

        if stop_stagnation and iter_sans_amelioration >= max_iter_sans_amelioration:
            print(f"Stagnation atteinte à l'itération {iteration}, arrêt prématuré.")
            break

        taille_liste_tabou = 10 if iteration % 15 == 0 else 15

        if len(var_couts) > 10:
            variance = np.var(var_couts[-10:])
            if variance < seuil_variance:
                break

    return meilleure_solution, meilleur_cout

# Paramètres de l'algorithme
nb_villes = 101
nb_iterations_max = 1000
taille_liste_tabou_initiale = 20
diversifier = True
intensifier = True
stop_stagnation = True
max_iter_sans_amelioration = 30
initialize_randomly = False
seuil_variance = 1e-4

G = build_graph_from_distance_matrix(distance_matrix)
start_time = time.time()

meilleure_solution, cout_meilleure_solution = recherche_tabou(G, nb_villes, nb_iterations_max, taille_liste_tabou_initiale, distance_matrix, diversifier, intensifier, stop_stagnation, max_iter_sans_amelioration, initialize_randomly, seuil_variance)

end_time = time.time()
elapsed_time = end_time - start_time

print("Meilleure solution trouvée:")
print(meilleure_solution)
print("Coût de la meilleure solution:")
print(cout_meilleure_solution)
print(f"Temps d'exécution: {elapsed_time} secondes")


Stagnation atteinte à l'itération 35, arrêt prématuré.
Meilleure solution trouvée:
[49, 32, 80, 50, 8, 70, 64, 34, 33, 77, 78, 2, 76, 75, 67, 79, 28, 23, 24, 54, 38, 66, 22, 55, 74, 40, 21, 73, 72, 20, 71, 3, 53, 11, 25, 27, 52, 39, 57, 12, 1, 56, 14, 42, 41, 86, 96, 91, 58, 36, 97, 99, 90, 13, 43, 37, 85, 15, 60, 84, 92, 4, 83, 16, 44, 7, 45, 35, 48, 46, 47, 18, 61, 9, 10, 63, 62, 89, 31, 65, 19, 29, 69, 30, 87, 6, 81, 82, 17, 59, 98, 95, 94, 93, 5, 88, 51, 100, 26, 68, 0, 0]
Coût de la meilleure solution:
697.9908329829229
Temps d'exécution: 42.68150544166565 secondes


In [ ]:
import random
import networkx as nx
import numpy as np
import time
from networkx.algorithms.matching import min_weight_matching

# Define the distance matrix
# (Your distance matrix goes here)

def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G

def distance(city1, city2):
    return distance_matrix[city1][city2]

def evaluate_solution(solution):
    return sum(distance(solution[i], solution[i + 1]) for i in range(len(solution) - 1)) + distance(solution[-1], solution[0])

def nearest_neighbor(cities):
    unvisited = cities[:]
    solution = [unvisited.pop(0)]
    while unvisited:
        nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
        solution.append(nearest)
        unvisited.remove(nearest)
    return solution

def christofides_tsp(distance_matrix):
    G = build_graph_from_distance_matrix(distance_matrix)
    return christofides_algorithm(G)

def two_opt(solution):
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(1, len(solution) - 1):
            for j in range(i + 1, len(solution)):
                if j - i == 1: continue
                new_solution = solution[:i] + solution[i:j][::-1] + solution[j:]
                if evaluate_solution(new_solution) < evaluate_solution(best):
                    best = new_solution
                    improved = True
    return best

def insertion_heuristic(cities):
    solution = [cities.pop(0)]
    while cities:
        best_increase = float('inf')
        best_city = None
        best_position = None
        for city in cities:
            for i in range(len(solution) + 1):
                new_solution = solution[:i] + [city] + solution[i:]
                increase = evaluate_solution(new_solution) - evaluate_solution(solution)
                if increase < best_increase:
                    best_increase = increase
                    best_city = city
                    best_position = i
        solution.insert(best_position, best_city)
        cities.remove(best_city)
    return solution

def improved_nearest_neighbor(cities):
    best_solution = None
    best_distance = float('inf')
    for start in range(len(cities)):
        unvisited = cities[:]
        solution = [unvisited.pop(start)]
        while unvisited:
            nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
            solution.append(nearest)
            unvisited.remove(nearest)
        total_distance = evaluate_solution(solution)
        if total_distance < best_distance:
            best_solution = solution
            best_distance = total_distance
    return best_solution

heuristics = {
    "nearest_neighbor": nearest_neighbor,
    "christofides_tsp": lambda cities: christofides_tsp(distance_matrix),
    "insertion_heuristic": insertion_heuristic,
    "improved_nearest_neighbor": improved_nearest_neighbor
}

scores = {name: 1 for name in heuristics}
uses = {name: 0 for name in heuristics}
used_heuristics = []

def select_heuristic(scores, iteration):
    global used_heuristics
    if iteration < len(heuristics):
        heuristic = list(heuristics.keys())[iteration]
    else:
        if len(used_heuristics) == len(heuristics):
            used_heuristics = []

        remaining_heuristics = [h for h in heuristics if h not in used_heuristics]
        heuristic = random.choices(remaining_heuristics, weights=[scores[h] for h in remaining_heuristics])[0]
        used_heuristics.append(heuristic)

    return heuristic

def update_scores(selected_heuristic, improvement):
    if improvement:
        scores[selected_heuristic] += 1
    else:
        scores[selected_heuristic] = max(1, scores[selected_heuristic] - 0.5)  # Penalize less to maintain exploration

def generer_solution_initiale(nb_villes):
    return random.sample(range(nb_villes), nb_villes)

def generer_voisinage(S):
    voisinage = []
    for i in range(1, len(S)):
        for j in range(i, len(S)):
            voisin = S[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

def recherche_locale_2opt(adjacency_matrix, route):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route) - 1):
                current_distance = adjacency_matrix[route[i]][route[i + 1]] + \
                                   adjacency_matrix[route[j]][route[j + 1]]
                new_distance = adjacency_matrix[route[i]][route[j]] + \
                               adjacency_matrix[route[i + 1]][route[j + 1]]
                if new_distance < current_distance:
                    route[i + 1:j + 1] = route[i + 1:j + 1][::-1]
                    improved = True
    return route

def christofides_algorithm(G):
    T = nx.minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)
    matching = min_weight_matching(odd_graph)
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']
    return hamiltonian_circuit, total_cost

def calculer_cout(route, matrice_distances):
    cout_total = 0
    for i in range(len(route) - 1):
        distance = matrice_distances[route[i]][route[i + 1]]
        cout_total += distance
    cout_total += matrice_distances[route[-1]][route[0]]
    return cout_total

def recherche_tabou(G, nb_villes, nb_iterations_max, taille_liste_tabou, matrice_distances, stop_stagnation, max_iter_sans_amelioration, seuil_variance):

    S, meilleur_cout = christofides_algorithm(G)
    S = recherche_locale_2opt(adjacency_matrix=matrice_distances, route=S)

    meilleure_solution = S[:]
    liste_tabou = []
    iter_sans_amelioration = 0
    var_couts = []

    for iteration in range(nb_iterations_max):
        selected_heuristic = select_heuristic(scores, iteration)
        S_prime = heuristics[selected_heuristic](S.copy())

        if isinstance(S_prime[0], list):
            S_prime = S_prime[0]

        S_prime = recherche_locale_2opt(matrice_distances, S_prime)
        cout_S_prime = calculer_cout(S_prime, matrice_distances)
        print(cout_S_prime, selected_heuristic)

        if (cout_S_prime < meilleur_cout and (S_prime not in liste_tabou or cout_S_prime < meilleur_cout)):
            meilleure_solution = S_prime[:]
            meilleur_cout = cout_S_prime
            update_scores(selected_heuristic, True)
            iter_sans_amelioration = 0
            var_couts.append(cout_S_prime)
        else:
            update_scores(selected_heuristic, False)
            iter_sans_amelioration += 1

        S = S_prime
        liste_tabou.append(S)
        if len(liste_tabou) > taille_liste_tabou:
            liste_tabou.pop(0)


        if stop_stagnation and iter_sans_amelioration >= max_iter_sans_amelioration:
            print(f"Stagnation atteinte à l'itération {iteration}, arrêt prématuré.")
            break

        taille_liste_tabou = 7 if iteration % 15 == 0 else 15

        if len(var_couts) > 10:
            variance = np.var(var_couts[-10:])
            if variance < seuil_variance:
                break

    return meilleure_solution, meilleur_cout

# Paramètres de l'algorithme
nb_villes = 101
nb_iterations_max = 1000
taille_liste_tabou_initiale = 20
stop_stagnation = True
max_iter_sans_amelioration = 20
initialize_randomly = True
seuil_variance = 1e-4

G = build_graph_from_distance_matrix(distance_matrix)
start_time = time.time()

meilleure_solution, cout_meilleure_solution = recherche_tabou(G, nb_villes, nb_iterations_max, taille_liste_tabou_initiale, distance_matrix, stop_stagnation, max_iter_sans_amelioration,seuil_variance)

end_time = time.time()
elapsed_time = end_time - start_time

print("Meilleure solution trouvée:")
print(meilleure_solution)
print("Coût de la meilleure solution:")
print(cout_meilleure_solution)
print(f"Temps d'exécution: {elapsed_time} secondes")



703.3912502525053 nearest_neighbor
677.7763631349077 christofides_tsp
689.1934123821978 insertion_heuristic
693.3511891643202 improved_nearest_neighbor
676.8052342754021 improved_nearest_neighbor
711.5843495874992 nearest_neighbor
677.7763631349077 christofides_tsp
689.1934123821978 insertion_heuristic
677.7763631349077 christofides_tsp
688.5011542693834 improved_nearest_neighbor
728.6842797764738 nearest_neighbor
692.401707461722 insertion_heuristic
698.209306088256 improved_nearest_neighbor
677.7763631349077 christofides_tsp
689.1934123821978 insertion_heuristic
713.0908610610511 nearest_neighbor
712.985840254411 nearest_neighbor
702.8534619400232 insertion_heuristic
677.7763631349077 christofides_tsp
688.5011542693834 improved_nearest_neighbor
677.7763631349077 christofides_tsp
688.5011542693834 improved_nearest_neighbor
728.6842797764738 nearest_neighbor
692.401707461722 insertion_heuristic
744.3276348701412 nearest_neighbor
Stagnation atteinte à l'itération 24, arrêt prématuré.
Me

In [1]:
  /**--------------------------------------------------------**/
  /**       C o n v e r s i o n   Z vers C (Standard)        **/
  /**             Realisee par Pr D.E ZEGOUR                 **/
  /**             E S I - Alger                              **/
  /**             Copywrite 2014                             **/
  /**--------------------------------------------------------**/

  #include <stdio.h>
  #include <stdlib.h>

  typedef int bool ;

  #define True 1
  #define False 0

  /** Implementation **\: ARBRE BINAIRE DE STRUCTURES**/

  /** Structures statiques **/

  typedef struct Tib Type_Tib  ;
  typedef Type_Tib * Typestr_Tib ;
  typedef int Type1_Tib  ;
  typedef bool Type2_Tib  ;
  struct Tib
    {
      Type1_Tib Champ1 ;
      Type2_Tib Champ2 ;
    };

  Type1_Tib Struct1_Tib ( Typestr_Tib S)
    {
      return  S->Champ1 ;
    }

  Type2_Tib Struct2_Tib ( Typestr_Tib S)
    {
      return  S->Champ2 ;
    }

  void Aff_struct1_Tib ( Typestr_Tib S, Type1_Tib Val )
    {
       S->Champ1 = Val ;
    }

  void Aff_struct2_Tib ( Typestr_Tib S, Type2_Tib Val )
    {
       S->Champ2 = Val ;
    }


  /** Arbres de recherche binaire **/

  typedef Typestr_Tib Typeelem_ATib   ;
  typedef struct Noeud_ATib * Pointeur_ATib ;

  struct Noeud_ATib
    {
      Typeelem_ATib  Val ;
      Pointeur_ATib Fg ;
      Pointeur_ATib Fd ;
      Pointeur_ATib Pere ;
     } ;

  Typeelem_ATib Info_ATib( Pointeur_ATib P )
    { return P->Val;   }

  Pointeur_ATib Fg_ATib( Pointeur_ATib P)
    { return P->Fg ; }

  Pointeur_ATib Fd_ATib( Pointeur_ATib P)
    { return P->Fd ; }

  Pointeur_ATib Pere_ATib( Pointeur_ATib P)
    { return P->Pere ; }

  void Aff_info_ATib ( Pointeur_ATib P, Typeelem_ATib Val)
    {
      Typeelem_ATib _Temp ;
      _Temp = malloc(sizeof(Type_Tib));
      /* Affectation globale de structure */
      _Temp->Champ1 = Val->Champ1;
      _Temp->Champ2 = Val->Champ2;
      Val = _Temp ;
       P->Val = Val ;
    }

  void Aff_fg_ATib( Pointeur_ATib P, Pointeur_ATib Q)
    { P->Fg =  Q;  }

  void Aff_fd_ATib( Pointeur_ATib P, Pointeur_ATib Q)
    { P->Fd =  Q ; }

  void Aff_pere_ATib( Pointeur_ATib P, Pointeur_ATib Q)
    { P->Pere =  Q ; }

  void Creernoeud_ATib( Pointeur_ATib *P)
    {
      *P = (struct Noeud_ATib *) malloc( sizeof( struct Noeud_ATib))   ;
      (*P)->Val = malloc(sizeof(Type_Tib));
      (*P)->Fg = NULL;
      (*P)->Fd = NULL;
      (*P)->Pere = NULL;
    }

  void Liberernoeud_ATib( Pointeur_ATib P)
    { free( P ) ; }

  /** Pour les variables temporaires **/
  typedef Typestr_Tib Typeelem_V4Tib;
  typedef Typeelem_V4Tib Typetab_V4Tib[4];


  /** Implementation **\: PILE DE ARBRES BINAIRES DE STRUCTURES**/
  /** Piles **/

  typedef Pointeur_ATib Typeelem_PATib ;
  typedef struct Maillon_PATib * Pointeur_PATib ;
  typedef   Pointeur_PATib  Typepile_PATib  ;

  struct Maillon_PATib
    {
      Typeelem_PATib  Val ;
      Pointeur_PATib Suiv ;
    } ;

  void Creerpile_PATib( Pointeur_PATib *P )
    { *P = NULL ; }

  bool Pilevide_PATib ( Pointeur_PATib P )
    { return  (P == NULL ); }

  void Empiler_PATib ( Pointeur_PATib *P,  Typeelem_PATib Val )
    {
      Pointeur_PATib Q;

      Q = (struct Maillon_PATib *) malloc( sizeof( struct Maillon_PATib))   ;
      Q->Val = Val ;
      Q->Suiv = *P;
      *P = Q;
    }

  void Depiler_PATib ( Pointeur_PATib *P,  Typeelem_PATib *Val )
    {
      Pointeur_PATib Sauv;

      if (! Pilevide_PATib (*P) )
        {
          *Val = (*P)->Val ;
          Sauv = *P;
          *P = (*P)->Suiv;
          free(Sauv);
        }
      else printf ("%s \n", "Pile vide");
    }


  /** Variables du programme principal **/
  Pointeur_ATib A=NULL;
    Typetab_V4Tib T_A;
  Type_Tib S_A;
  int _Izw;  /** Variable de contrôle **/

  /** Macro operations **/


  /** Creation d'un arbre de recherche binaire **/

  void Creer_arb_ATib ( Pointeur_ATib *Arbre, Typetab_V4Tib Tab , int  N)
    {
      int I;
      Pointeur_ATib P, Q ;

      /* Creation De La Racine */
      Creernoeud_ATib ( &P ) ;
      *Arbre = P ;
      Aff_info_ATib ( P , Tab[0] ) ;
      for(I=2;I<=N;++I)
        {
          P = *Arbre ;
          Q = *Arbre ;
          while ((  Tab[I-1]->Champ1 != Struct1_Tib(Info_ATib ( P ) )) && ( Q != NULL ))
            {
              P = Q ;
              if ( Tab[I-1]->Champ1 <  Struct1_Tib(Info_ATib ( P ) ))
                Q = Fg_ATib ( P );
              else Q = Fd_ATib ( P ) ;
            }
          if ( Tab[I-1]->Champ1 == Struct1_Tib(Info_ATib ( P ) ))
            printf ("%s \n", "Un double existe");
          else
            {
              Creernoeud_ATib ( &Q ) ;
              Aff_info_ATib ( Q , Tab[I-1] ) ;
              if ( Tab[I-1]->Champ1 <  Struct1_Tib(Info_ATib ( P ) ))
                Aff_fg_ATib ( P , Q ) ;
              else Aff_fd_ATib ( P , Q );
              Aff_pere_ATib (Q, P);
            }
        }
    }


  /** Prototypes des fonctions **/

  void Traversal_ll_lr (Pointeur_ATib *A);
  void Traversal_ll_lr (Pointeur_ATib *A)
    {
      /** Variables locales **/
      Pointeur_ATib R=NULL;
      Pointeur_ATib Tmp=NULL;
      Pointeur_ATib Racine=NULL;
      Pointeur_PATib P=NULL;
      Typestr_Tib V;
      Type_Tib _Struct_Temp1;

      /** Corps du module **/
     V = malloc(sizeof(Type_Tib));
     if( ( *A != NULL )) {
       Racine  =  *A ;
       R  =  Racine ;
       Creerpile_PATib (& P ) ;
       while( ( ( R != NULL ) && ( ! Pilevide_PATib ( P ) ) ))  {
         while( ( ( R != NULL ) && ( ! Struct2_Tib ( Info_ATib ( R )  ) ) ))  {
           Empiler_PATib (& P , R ) ;
           R  =  Fg_ATib ( R ) ;

         } ;
         Depiler_PATib (& P , &R ) ;
        /* Vérifier si le noeud est une feuille non déja visitée */
         if( ( ( Fg_ATib ( R ) == NULL ) || ( Struct2_Tib ( Info_ATib ( Fg_ATib ( R ) )  ) ) ) && ( ( Fd_ATib ( R ) == NULL ) || ( Struct2_Tib ( Info_ATib ( Fd_ATib ( R ) )  ) ) )) {
           /** Ecriture d'une structure */
           _Struct_Temp1 = *Info_ATib(R);
           printf ( " %d", _Struct_Temp1.Champ1 );
           printf ( " %d", _Struct_Temp1.Champ2 ) ;
      /* Affectation globale de structure */
           V->Champ1 =   Info_ATib ( R )->Champ1;
           V->Champ2 =   Info_ATib ( R )->Champ2;
 ;
           Aff_struct2_Tib ( V  , True ) ;
           Aff_info_ATib ( R , V ) ;

         } ;
         if( ( Fd_ATib ( R ) != NULL ) && ( ! Struct2_Tib ( Info_ATib ( Fd_ATib ( R ) )  ) )) {
           R  =  Fd_ATib ( R ) ;
           }
         else
           {
           Tmp  =  R ;
           Depiler_PATib (& P , &R ) ;
           while( ( ( Fd_ATib ( R ) == NULL ) || ( Struct2_Tib ( Info_ATib ( Fd_ATib ( R ) )  ) ) || ( Tmp == Fd_ATib ( R ) ) ) && ( ! Pilevide_PATib ( P ) ))  {
             Tmp  =  R ;
             if( ( ! Pilevide_PATib ( P ) )) {
               Depiler_PATib (& P , &R ) ;

             } ;

           } ;
           if( ( Pilevide_PATib ( P ) ) && ( ! Struct2_Tib ( Info_ATib ( Racine )  ) )) {
             Empiler_PATib (& P , Racine ) ;
             }
           else
             {
             if( ( ! Pilevide_PATib ( P ) )) {
               R  =  Fd_ATib ( R ) ;
               }
             else
               {
               R  =  NULL ;

             } ;

           } ;

         } ;

       } ;

     } ;

    }

  int main(int argc, char *argv[])
    {
     int _Izw2;for (_Izw2=0; _Izw2<4; ++_Izw2)
       T_A[_Izw2] = malloc(sizeof(Type_Tib));
     S_A.Champ1 = 1 ;
     S_A.Champ2 = False ;
     *T_A [ 0 ] = S_A ;
     S_A.Champ1 = 7 ;
     S_A.Champ2 = False ;
     *T_A [ 1 ] = S_A ;
     S_A.Champ1 = 11 ;
     S_A.Champ2 = False ;
     *T_A [ 2 ] = S_A ;
     S_A.Champ1 = 5 ;
     S_A.Champ2 = False ;
     *T_A [ 3 ] = S_A ;
     Creer_arb_ATib (&A , T_A , 4 )  ;
     Traversal_ll_lr ( & A ) ;


      system("PAUSE");
      return 0;
    }


SyntaxError: invalid syntax (<ipython-input-1-1117a9615112>, line 1)

In [ ]:
import random
import networkx as nx
import numpy as np
import time
from networkx.algorithms.matching import min_weight_matching

# Define the distance matrix
# (Your distance matrix goes here)

def build_graph_from_distance_matrix(distance_matrix):
    G = nx.Graph()
    for i in range(len(distance_matrix)):
        for j in range(i + 1, len(distance_matrix)):
            G.add_edge(i, j, weight=distance_matrix[i][j])
    return G

def distance(city1, city2):
    return distance_matrix[city1][city2]

def evaluate_solution(solution):
    return sum(distance(solution[i], solution[i + 1]) for i in range(len(solution) - 1)) + distance(solution[-1], solution[0])

def nearest_neighbor(cities):
    unvisited = cities[:]
    solution = [unvisited.pop(0)]
    while unvisited:
        nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
        solution.append(nearest)
        unvisited.remove(nearest)
    return solution

def christofides_tsp(distance_matrix):
    G = build_graph_from_distance_matrix(distance_matrix)
    return christofides_algorithm(G)

def two_opt(solution):
    best = solution
    improved = True
    while improved:
        improved = False
        for i in range(1, len(solution) - 1):
            for j in range(i + 1, len(solution)):
                if j - i == 1: continue
                new_solution = solution[:i] + solution[i:j][::-1] + solution[j:]
                if evaluate_solution(new_solution) < evaluate_solution(best):
                    best = new_solution
                    improved = True
    return best

def insertion_heuristic(cities):
    solution = [cities.pop(0)]
    while cities:
        best_increase = float('inf')
        best_city = None
        best_position = None
        for city in cities:
            for i in range(len(solution) + 1):
                new_solution = solution[:i] + [city] + solution[i:]
                increase = evaluate_solution(new_solution) - evaluate_solution(solution)
                if increase < best_increase:
                    best_increase = increase
                    best_city = city
                    best_position = i
        solution.insert(best_position, best_city)
        cities.remove(best_city)
    return solution

def improved_nearest_neighbor(cities):
    best_solution = None
    best_distance = float('inf')
    for start in range(len(cities)):
        unvisited = cities[:]
        solution = [unvisited.pop(start)]
        while unvisited:
            nearest = min(unvisited, key=lambda city: distance(solution[-1], city))
            solution.append(nearest)
            unvisited.remove(nearest)
        total_distance = evaluate_solution(solution)
        if total_distance < best_distance:
            best_solution = solution
            best_distance = total_distance
    return best_solution

heuristics = {
    "nearest_neighbor": nearest_neighbor,
    "christofides_tsp": lambda cities: christofides_tsp(distance_matrix),
    "insertion_heuristic": insertion_heuristic,
    "improved_nearest_neighbor": improved_nearest_neighbor
}

scores = {name: 1 for name in heuristics}
uses = {name: 0 for name in heuristics}
used_heuristics = []

def select_heuristic(scores, iteration):
    global used_heuristics
    if iteration < len(heuristics):
        heuristic = list(heuristics.keys())[iteration]
    else:
        if len(used_heuristics) == len(heuristics):
            used_heuristics = []

        remaining_heuristics = [h for h in heuristics if h not in used_heuristics]
        heuristic = random.choices(remaining_heuristics, weights=[scores[h] for h in remaining_heuristics])[0]
        used_heuristics.append(heuristic)

    return heuristic

def update_scores(selected_heuristic, improvement):
    if improvement:
        scores[selected_heuristic] += 1
    else:
        scores[selected_heuristic] = max(1, scores[selected_heuristic] - 0.5)  # Penalize less to maintain exploration

def generer_solution_initiale(nb_villes):
    return random.sample(range(nb_villes), nb_villes)

def generer_voisinage(S):
    voisinage = []
    for i in range(1, len(S)):
        for j in range(i, len(S)):
            voisin = S[:]
            voisin[i:j] = voisin[i:j][::-1]
            voisinage.append(voisin)
    return voisinage

def recherche_locale_2opt(adjacency_matrix, route):
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route) - 1):
                current_distance = adjacency_matrix[route[i]][route[i + 1]] + \
                                   adjacency_matrix[route[j]][route[j + 1]]
                new_distance = adjacency_matrix[route[i]][route[j]] + \
                               adjacency_matrix[route[i + 1]][route[j + 1]]
                if new_distance < current_distance:
                    route[i + 1:j + 1] = route[i + 1:j + 1][::-1]
                    improved = True
    return route

def christofides_algorithm(G):
    T = nx.minimum_spanning_tree(G, weight='weight')
    odd_degree_nodes = [node for node in T.nodes() if T.degree(node) % 2 == 1]
    odd_graph = G.subgraph(odd_degree_nodes)
    matching = min_weight_matching(odd_graph)
    multigraph = nx.MultiGraph(T)
    for edge in matching:
        multigraph.add_edge(edge[0], edge[1], weight=G[edge[0]][edge[1]]['weight'])
    eulerian_circuit = list(nx.eulerian_circuit(multigraph))
    visited = set()
    hamiltonian_circuit = []
    total_cost = 0
    last_node = None
    for u, v in eulerian_circuit:
        if u not in visited:
            if last_node is not None:
                total_cost += G[last_node][u]['weight']
            visited.add(u)
            hamiltonian_circuit.append(u)
            last_node = u
        if v not in visited:
            if last_node is not None:
                total_cost += G[last_node][v]['weight']
            visited.add(v)
            hamiltonian_circuit.append(v)
            last_node = v
    if hamiltonian_circuit:
        start_node = hamiltonian_circuit[0]
        end_node = hamiltonian_circuit[-1]
        if start_node != end_node:
            hamiltonian_circuit.append(start_node)
            total_cost += G[end_node][start_node]['weight']
    return hamiltonian_circuit, total_cost

def calculer_cout(route, matrice_distances):
    cout_total = 0
    for i in range(len(route) - 1):
        distance = matrice_distances[route[i]][route[i + 1]]
        cout_total += distance
    cout_total += matrice_distances[route[-1]][route[0]]
    return cout_total

def recherche_tabou(G, nb_villes, nb_iterations_max, taille_liste_tabou, matrice_distances, stop_stagnation, max_iter_sans_amelioration, seuil_variance):


    S, meilleur_cout = christofides_algorithm(G)  # Apply Christofides once
    S = recherche_locale_2opt(adjacency_matrix=matrice_distances, route=S)

    meilleure_solution = S[:]
    liste_tabou = [tuple(S)]  # Add initial solution to tabu list
    iter_sans_amelioration = 0
    var_couts = []

    for iteration in range(nb_iterations_max):
        selected_heuristic = select_heuristic(scores, iteration)
        S_prime = heuristics[selected_heuristic](S.copy())

        if isinstance(S_prime[0], list):
            S_prime = S_prime[0]

        S_prime = recherche_locale_2opt(matrice_distances, S_prime)
        cout_S_prime = calculer_cout(S_prime, matrice_distances)
        #print(cout_S_prime, selected_heuristic)

        if (cout_S_prime < meilleur_cout and (tuple(S_prime) not in liste_tabou or cout_S_prime < meilleur_cout)):
            meilleure_solution = S_prime[:]
            meilleur_cout = cout_S_prime
            update_scores(selected_heuristic, True)
            iter_sans_amelioration = 0
            var_couts.append(cout_S_prime)
        else:
            update_scores(selected_heuristic, False)
            iter_sans_amelioration += 1

        S = S_prime
        liste_tabou.append(tuple(S))
        if len(liste_tabou) > taille_liste_tabou:
            liste_tabou.pop(0)

        if stop_stagnation and iter_sans_amelioration >= max_iter_sans_amelioration:
            print(f"Stagnation atteinte à l'itération {iteration}, arrêt prématuré.")
            break

        taille_liste_tabou = 7 if iteration % 15 == 0 else 15

        if len(var_couts) > 10:
            variance = np.var(var_couts[-10:])
            if variance < seuil_variance:
                break

    return meilleure_solution, meilleur_cout

# Parameters and execution
nb_villes = 101
nb_iterations_max = 1000
taille_liste_tabou_initiale = 20
stop_stagnation = True
max_iter_sans_amelioration = 20
initialize_randomly = True
seuil_variance = 1e-4

G = build_graph_from_distance_matrix(distance_matrix)
start_time = time.time()

meilleure_solution, cout_meilleure_solution = recherche_tabou(G, nb_villes, nb_iterations_max, taille_liste_tabou_initiale, distance_matrix, stop_stagnation, max_iter_sans_amelioration, seuil_variance)

end_time = time.time()
elapsed_time = end_time - start_time

print("Meilleure solution trouvée:")
print(meilleure_solution)
print("Coût de la meilleure solution:")
print(cout_meilleure_solution)
print(f"Temps d'exécution: {elapsed_time} secondes")


Stagnation atteinte à l'itération 21, arrêt prématuré.
Meilleure solution trouvée:
[0, 49, 75, 76, 67, 79, 11, 25, 20, 72, 71, 73, 21, 40, 74, 22, 66, 38, 55, 3, 24, 54, 53, 23, 28, 2, 78, 77, 33, 80, 32, 50, 8, 34, 70, 64, 65, 19, 29, 69, 30, 87, 61, 18, 10, 9, 31, 89, 62, 63, 48, 35, 46, 47, 6, 81, 7, 45, 44, 82, 59, 4, 83, 16, 85, 37, 13, 43, 15, 60, 92, 58, 97, 84, 90, 99, 36, 41, 42, 14, 56, 1, 86, 96, 91, 94, 95, 98, 17, 51, 88, 5, 93, 12, 57, 39, 52, 100, 27, 26, 68, 0]
Coût de la meilleure solution:
677.7763631349077
Temps d'exécution: 33.290839433670044 secondes
